# Import Library

In [1]:
import numpy as np
import pandas as pd
import math
import random
import dill
try:
    dill.load_session('Pengujian_Normalisasi.db')
except Exception as ex:
    print("Data Belum Tersimpan!")

# Support

## Identify Label

In [2]:
def count_label(data):
    distinct = [data[0][-1]]
    for i in range(len(data)):
        check = True
        for j in range(len(distinct)):
            if distinct[j] == data[i][-1]:
                check = False
        if check:
            distinct.append(data[i][-1])
    return np.array(distinct).astype(int), len(distinct)

## Model Evaluation Performance

### Mean + STD

In [3]:
def evaluation(arr_acc):
    mean = np.mean(arr_acc)
    varian = sum((arr_acc - mean) * (arr_acc - mean)) / len(arr_acc)
    std = math.sqrt(varian)
    return mean, std

### Precision, Recall dan F1-Score

In [4]:
def toNumpyArray(coff):
    coff = np.array(coff)
    for i in range(len(coff)):
        coff[i] = np.array(coff[i])
        for j in range(len(coff[i])):
            coff[i][j] = np.array(coff[i][j])
    return coff

def pickCoff(coff):
    coff = toNumpyArray(coff)
    choosen_coff = []
    for i in range(len(coff)):
        for j in range(len(coff[i])):
            acc = 0
            for k in range(len(coff[i][j])):
                for l in range(len(coff[i][j][k])):
                    if k == l:
                        acc += coff[i][j][k][l]
            acc /= np.sum(coff[i][j])
            if acc != 1:
                choosen_coff.append(coff[i][j])
                break
            elif j == len(coff[i]) - 1:
                choosen_coff.append(coff[i][j])
    return np.array(choosen_coff)

def calcPrecRec(coff):
    coff = pickCoff(coff)
    tfn = []
    tfp = []
    ttp = []
    precision = []
    recall = []
    for i in range(len(coff)):
        tfn.append(np.sum(coff[i], axis = 1))
        tfp.append(np.sum(coff[i], axis = 0))
        ttp_count = 0
        for j in range(len(coff[i])):
            for k in range(len(coff[i][j])):
                if j == k:
                    tfn[i][j] -= coff[i][j][k]
                    tfp[i][j] -= coff[i][j][k]
                    ttp_count += coff[i][j][k]
        ttp.append(ttp_count)
    for i in range(len(tfn)):
        precision.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        recall.append(np.array([0 for i in range(len(tfn[i]))]).astype(np.float32))
        for j in range(len(tfn[i])):
            precision[i][j] = ttp[i]/(ttp[i] + tfp[i][j])
            recall[i][j] = ttp[i]/(ttp[i] + tfn[i][j])
    return np.array(precision), np.array(recall)

def calcF1Score(precision, recall):
    f1 = []
    for i in range(len(precision)):
        prec = np.mean(precision[i])
        rec = np.mean(recall[i])
        f1.append(2*prec*rec/(prec+rec))
    return np.array(f1)

## Euclidian Distance

In [5]:
def euclidian(X, Y, categories = []):
    euc = 0
    for i in range(len(X)):
        if not i in categories: ## similarity matrix
            euc += math.pow((X[i]-Y[i]), 2)
    euc = math.sqrt(euc)
    ##------------Similiratity Matrix-------------##
    if len(categories) != 0:
        p = len(categories)
        m = 0
        for i in range(len(categories)):
            if X[categories[i]] == Y[categories[i]]:
                m += 1
        cat_dist = (p-m)/p
        euc += cat_dist
    ##--------------------------------------------##
    return euc

# Pre Processing

## Random Data

In [6]:
def randomize(arr):
    for i in range(len(arr)-1, 0, -1):
        j = random.randint(0, i)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

## Normalisasi

In [7]:
def normalize(data):
    data = np.array(data).astype(float)
    max_value = np.max(data, axis=0)
    min_value = np.min(data, axis=0)
    for i in range(len(data)):
        for j in range(len(data[0])-1):
            data[i][j] = (data[i][j] - min_value[j]) / (max_value[j] - min_value[j])
    return data

## Missing Value

In [8]:
def handleMissValue(data):
    filled_data = np.array([[0 for i in range(len(data[0]))] for j in range(len(data))])
    cLabel = count_label(data)[1]
    arr_data = [[] for j in range(cLabel)]
    mean = []
    for i in range(len(data)):
        arr_data[int(data[i][-1])-1].append(data[i])
    counter = 0
    for i in range(len(arr_data)):
        mean.append(np.nanmean(arr_data[i], axis=0))
        for j in range(len(arr_data[i])):
            for k in range(len(arr_data[i][j])):
                if np.isnan(arr_data[i][j][k]):
                    arr_data[i][j][k] = mean[i][k]
            filled_data[counter] = arr_data[i][j]
            counter += 1
    return filled_data

## Ekstraksi File 


In [9]:
def extraction(file, is_normal = True, is_random = True):
    data = pd.read_excel(file)
    data = handleMissValue(np.array(data))
    data = pd.DataFrame(data)
    data.drop_duplicates()
    data = np.array(data)
    if is_normal:
        data = normalize(data)
    if is_random:
        data = randomize(data.tolist())
    return data

# LMKHNCN

## Set of KNCN

### 1 NCN

In [10]:
def closest(Xx, Xx_label, Y, cLabel, nominal):
    distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
    min_distance_value = [0 for i in range(cLabel)]
    min_distance_index = [0 for i in range(cLabel)]
    for i in range(len(Xx)):
        distance[((Xx_label[i][0])-1)][i] = euclidian(Xx[i], Y, nominal)
    for i in range(cLabel):
        min_distance_value[i] = min(x for x in distance[i] if x is not None)
        min_distance_index[i] = distance[i].index(min(x for x in distance[i] if x is not None))
    return np.array(distance), min_distance_index, min_distance_value

### N NCN

In [11]:
def kncn(k, Xx, Xx_label, Y, cLabel, close_dist, nominal):
    ncn = np.array([[0 for i in range(k)] for j in range(cLabel)])
    for i in range(cLabel):
        ncn[i][0] = close_dist[i]

    for i in range (1, k):
        distance = [[None for i in range(len(Xx))] for j in range(cLabel)]
        min_distance_value = [0 for i in range(cLabel)]
        min_distance_index = [0 for i in range(cLabel)]
        for j in range(len(Xx)):
            label = Xx_label[j][0] - 1
            check = True
            for a in range(i):
                if ncn[label][a] == j:
                    check = False
            if check:
                add = 0 + Xx[j]
                for a in range(i):
                    indeks_ncn = ncn[label][a]
                    add += Xx[indeks_ncn]
                centroid = add / (i+1)
                distance[label][j] = euclidian(Y, centroid, nominal)
        for j in range(cLabel):
            min_distance_value[j] = min(x for x in distance[j] if x is not None)
            min_distance_index[j] = distance[j].index(min(x for x in distance[j] if x is not None))
            ncn[j][i] = min_distance_index[j]
    return ncn

## Local Mean Vector

In [12]:
def localMeanVector(Xx, ncn):
    lm_ncn = [[0 for i in range(len(ncn[0]))] for j in range(len(ncn))]
    for i in range(len(ncn)):
        lm_ncn[i][0] = Xx[ncn[i][0]]
        for j in range(1, len(ncn[0])):
            add = 0 + lm_ncn[i][0]
            for a in range(j):
                add += Xx[ncn[i][a + 1]]
            lm_ncn[i][j] = add / (j+1)
    return np.array(lm_ncn)

## Harmonic Mean Distance

In [13]:
def harmonicMean(lm_ncn, Y, nominal):
    k = len(lm_ncn[0])
    hm_ncn = []
    for i in range(len(lm_ncn)):
        add = 0
        for j in range(len(lm_ncn[0])):
            distance = euclidian(Y, lm_ncn[i][j], nominal)
            ## Centroid Lokal i Tepat Pada Data Uji
            if distance == 0:
                add =+ 0
            else:
                add += (1/distance)
        ## Semua Centroid Lokal Tepat Pada Data Uji
        if add == 0:
            harmonic_mean = 0
        else:
            harmonic_mean = (k/add)
        hm_ncn.append(harmonic_mean)
    return hm_ncn

## Classification Decision

In [14]:
def decision(hm_ncn, Y_label):
    min_distance_index = hm_ncn.index(min(x for x in hm_ncn))
    return (min_distance_index + 1), Y_label[0]

# Performance Evaluation

## K Cross Fold Validation

In [15]:
def crossValidation(data, cross_val, cross_index, label_index):
    split_test = int(len(data) * (1 / cross_val))
    change_index = (cross_index - 1) * split_test
    
    for i in range(split_test):
        j = (i + change_index)
        data[i], data[j] = data[j], data[i]
    
    data_train_feature = pd.DataFrame(data[split_test:]).drop([label_index], axis=1)
    data_train_label = pd.DataFrame(pd.DataFrame(data[split_test:]), columns=[label_index]).astype(np.int)
    data_test_feature = pd.DataFrame(data[0:split_test]).drop([label_index], axis=1)
    data_test_label = pd.DataFrame(pd.DataFrame(data[0:split_test]), columns=[label_index]).astype(np.int)
    
    data_train_feature = np.array(data_train_feature)
    data_train_label = np.array(data_train_label)
    data_test_feature = np.array(data_test_feature)
    data_test_label = np.array(data_test_label)

    return data, data_train_feature, data_train_label, data_test_feature, data_test_label

## LMKHNCN

In [16]:
def lmkhncn(k, Xx, Xx_label, Yy, Yy_label, cLabel, nominal):
    acc = 0
    coff = np.array([[0 for i in range(jml_kelas)] for j in range(jml_kelas)])
    
    for i in range(len(Yy)):
        jarak, jarak_index_min, jarak_nilai_min = closest(Xx, Xx_label, Yy[i], cLabel, nominal)
        ncn = kncn(k, Xx, Xx_label, Yy[i], cLabel, jarak_index_min, nominal)
        local_mean = localMeanVector(Xx, ncn)
        harmonic_mean = harmonicMean(local_mean, Yy[i], nominal)
        predict, actual = decision(harmonic_mean, Yy_label[i])
        
        coff[actual-1][predict-1] += 1
        if(predict == actual):
            acc += 1
#         print(str(round((i+1)/len(Yy),2)*100)+"%. Prediksi :"+ str(predict) + " Actual :" + str(actual) + " Acc Lokal: "+ str(acc/(i+1)))
    
    acc /= len(Yy)
    return acc, coff

# Inisialisasi

In [17]:
k = 15
nominal = [8] ##Kawin Attribute
file = 'Dataset Pegawai.xlsx'
cross_val = 10
label_index = 9 #Kinerja
normalisasi = True
random_data = True
data = extraction(file, normalisasi, random_data)
arr_kelas, jml_kelas = count_label(data)

# Testing

In [ ]:
arr_coff = [[] for i in range(k)]
arr_mean_acc = [0 for i in range(k)]
arr_std_acc  = [0 for i in range(k)]

for a in range(1, k + 1): #Pengujian Parameter K
    arr_acc = [0 for i in range(cross_val)]
    for i in range(1, cross_val + 1):
        data, train_f, train_l, test_f, test_l = crossValidation(data, cross_val, i, label_index)
        acc, coff = lmkhncn(a, train_f, train_l, test_f, test_l, jml_kelas, nominal)
        arr_acc[i-1] = acc
        arr_coff[a-1].append(np.array(coff))
        print("Cross-"+str(i)+" Acc:"+ str(acc))
    mean, std = evaluation(arr_acc)
    arr_mean_acc[a-1] = mean
    arr_std_acc[a-1] = std
    print('K-{0} Mean : {1} with Standar Deviation : {2}.'.format(a,mean,std))

In [18]:
print(np.array(arr_mean_acc))
print(np.array(arr_std_acc))
print(np.array(arr_coff))

[0.95045593 0.9656535  0.96732523 0.9674772  0.96762918 0.96717325
 0.96641337 0.9662614  0.96504559 0.96474164 0.96352584 0.96246201
 0.96170213 0.9606383  0.9600304 ]
[0.00706973 0.00391614 0.00544786 0.00456939 0.00424717 0.00379635
 0.00415925 0.00503129 0.00465949 0.00464956 0.00480589 0.00548167
 0.0059173  0.00586437 0.00604283]
[[[[223  13   0]
   [ 16 318   0]
   [  0   1  87]]

  [[248  15   0]
   [ 17 299   2]
   [  0   3  74]]

  [[218  12   0]
   [ 19 316   1]
   [  0   2  90]]

  ...

  [[236  10   0]
   [ 16 310   2]
   [  0   2  82]]

  [[248  15   0]
   [ 19 288   1]
   [  0   2  85]]

  [[241   9   0]
   [ 23 299   5]
   [  0   5  76]]]


 [[[245   5   0]
   [ 14 311   2]
   [  0   3  78]]

  [[223  13   0]
   [ 11 323   0]
   [  0   1  87]]

  [[253  10   0]
   [  8 307   3]
   [  0   3  74]]

  ...

  [[245   7   0]
   [ 11 327   0]
   [  0   2  66]]

  [[238   8   0]
   [ 11 317   0]
   [  0   2  82]]

  [[250  13   0]
   [ 13 295   0]
   [  0   2  85]]]


 [[[250 

In [19]:
precision, recall = calcPrecRec(arr_coff)
f1Score = calcF1Score(precision,recall)
print(precision)
print(recall)
print(f1Score)

[[0.9751553  0.97819316 1.        ]
 [0.97839504 0.98753893 0.9968553 ]
 [0.9828927  0.97681606 1.        ]
 [0.99068326 0.9815385  0.996875  ]
 [0.98297215 0.9814529  1.        ]
 [0.9906687  0.97849464 0.9984326 ]
 [0.9845679  0.9845679  1.        ]
 [0.992224   0.9770291  1.        ]
 [0.98748046 0.972265   0.99841774]
 [0.990566   0.97072417 0.99526066]
 [0.9906103  0.9708589  1.        ]
 [0.98748046 0.97376543 0.9968404 ]
 [0.9937402  0.97094804 1.        ]
 [0.9953052  0.9739663  0.9968652 ]
 [0.98602486 0.97842836 1.        ]]
[[0.97971916 0.9751553  0.99841017]
 [0.9921753  0.9753846  0.9952904 ]
 [0.979845   0.9828927  0.9968454 ]
 [0.9845679  0.9876161  0.996875  ]
 [0.9844961  0.98297215 0.99686027]
 [0.98       0.98913044 0.9984326 ]
 [0.98608965 0.9845679  0.9984351 ]
 [0.9785276  0.992224   0.9984351 ]
 [0.97527045 0.9859375  0.9968404 ]
 [0.9737249  0.9859155  0.9968355 ]
 [0.97235024 0.9906103  0.99842274]
 [0.97829455 0.9843994  0.99526817]
 [0.97243494 0.9937402  0.9

# Save Data

In [20]:
# dill.dump_session('Pengujian_Normalisasi.db')